In [1]:
!pip install dash

     |████████████████████████████████| 7.3 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 9.3 MB/s  eta 0:00:01
     |████████████████████████████████| 25.3 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 51.2 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=409e3d735a6b2e63f08b9229a12fea9956ae82f54f269730e93c8304880cc713
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=ba30a0529344bfc108a4af733c320a27a34da11a7284c6bc78e65cac3034139f
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3897 sha256=8cb0b17ec317f58e464ce9d52bf8508908af90e620fb8f6dbe5a

In [3]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'spacex_launch_dash.csv'

In [2]:
# Import required libraries
import pandas as pd
import dash
# import dash_html_components as html
from dash import html

# import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                            id='site-dropdown',
                                            options=[
                                                # 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                            value='All',
                                            placeholder="Place holder here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')                                
                                        
                                        
                                        ),
                                html.Br(),

                                html.P("Payload range (Kg):"),

                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                    1000: '2500',
                                                    5000: '5000',
                                                    7500: '7500'
                                                    },
                                                value=[min_payload, max_payload])
                                        ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value')])


def build_pie(site):
    df = spacex_df
    if site == 'ALL':
        data = df.loc[df['class']==1]
        fig = px.pie(df,  values='class', names='Launch Site', title='Total Success Launches by site')
    #fig = px.pie(df, values='pop', names='country', title='Population of European continent')

    else:
        filtered_df= df[df['Launch Site'] == site]
        dff=filtered_df.groupby(['Launch Site', 'class']).size().reset_index()
        dff.rename(columns={0:'count'}, inplace=True)
        fig = px.pie(dff, names='class', values='count',
                    title='Total Success Launches for site ' + site)
    return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")
              ])

def get_scatter(entered_site,payload_range ):
    low,high = (payload_range[0], payload_range[1])
    df = spacex_df
    filtered_dfa = df[df['Payload Mass (kg)'].between(low,high)]
    if entered_site == 'ALL':    
        fig = px.scatter(filtered_dfa, y="class", x="Payload Mass (kg)", color="Booster Version Category" 
                         ,title="Correlation between Payload and Success for all Sites") 
        return fig
    else:
        filtered_dfb = filtered_dfa.loc[filtered_dfa['Launch Site']== entered_site]     
        fig = px.scatter(filtered_dfb, y="class", x="Payload Mass (kg)", color="Booster Version Category" 
                         ,title="Correlation between Payload and Success for " + entered_site )
        return fig



# Run the app
if __name__ == '__main__':
    app.run_server()
 

FileNotFoundError: [Errno 2] No such file or directory: 'spacex_launch_dash.csv'